In [11]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import model_selection
from sklearn import linear_model
from sklearn import metrics
import datetime
import yaml
import pickle
import requests
from pathlib import Path

In [2]:
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/taylor/cloud.data"
r = requests.get(url, allow_redirects=True)

open('cloud.data', 'wb').write(r.content)

209266

In [66]:
 with open("config.yaml", 'r') as f:
        config = yaml.load(f, Loader=yaml.FullLoader)

In [53]:
config['train_test_split']

{'test_size': 0.4, 'random_state': 423}

In [11]:
def load_data(filename, c1_start_ix, c1_end_ix, c2_start_ix, c2_end_ix):
    
    """Create dataframe from cloud.data with two cloud classes"""
    
    # Parse data file
    with open(filename,'r') as f:
        data = [[s for s in line.split(' ') if s!=''] for line in f.readlines()]
    
    # Create DataFrame for first type of cloud
    first_cloud = data[c1_start_ix:c1_end_ix]
    first_cloud = [[float(s.replace('/n', '')) for s in cloud]
                    for cloud in first_cloud]
    first_cloud = pd.DataFrame(first_cloud, columns=columns)
    first_cloud['class'] = np.zeros(len(first_cloud))
    
    # Create DataFrame for second type of cloud
    second_cloud = data[c2_start_ix:c2_end_ix]
    second_cloud = [[float(s.replace('/n', '')) for s in cloud]
                    for cloud in second_cloud]
    second_cloud = pd.DataFrame(second_cloud, columns=columns)
    second_cloud['class'] = np.ones(len(second_cloud))
    
    # Final DataFrame with both types of clouds
    data = pd.concat([first_cloud, second_cloud])
    
    return data

In [24]:
def create_features(features):
    
    """Create additional features"""
    
    features['visible_range'] = (features.visible_max - features.visible_min)
    
    features['visible_norm_range'] = (
        features.visible_max - features.visible_min).divide(features.visible_mean)

    features['log_entropy'] = features.visible_entropy.apply(np.log)

    features['entropy_x_contrast'] = features.visible_contrast.multiply(
        features.visible_entropy)

    features['IR_range']  = features.IR_max - features.IR_min

    features['IR_norm_range'] = (features.IR_max - features.IR_min).divide(
        features.IR_mean)
    
    return features

In [19]:
columns = ['visible_mean', 'visible_max', 'visible_min', 
           'visible_mean_distribution', 'visible_contrast', 
           'visible_entropy', 'visible_second_angular_momentum', 
           'IR_mean', 'IR_max', 'IR_min']

In [20]:
test = load_data(config['data_ingestion']['filename'], 
                config['data_ingestion']['class1_start_ix'],
                config['data_ingestion']['class1_end_ix'],
                config['data_ingestion']['class2_start_ix'],
                config['data_ingestion']['class2_end_ix'])

In [21]:
features = test[columns]
target = test["class"]

In [26]:
features = create_features(features)

In [55]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(
    features, target, **config['train_test_split'])

In [67]:
lr = linear_model.LogisticRegression(**config['logistic_regression'])

In [68]:
initial_features = ['visible_norm_range', 'log_entropy', 'IR_mean',
    'entropy_x_contrast','IR_norm_range', 'visible_mean']

In [69]:
lr.fit(X_train[initial_features], y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=False,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=423, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [76]:
ypred_proba_test = lr.predict_proba(X_test[initial_features])[:,1]
ypred_bin_test = lr.predict(X_test[initial_features])

In [77]:
auc = sklearn.metrics.roc_auc_score(y_test, ypred_proba_test)
confusion = sklearn.metrics.confusion_matrix(y_test, ypred_bin_test)
accuracy = sklearn.metrics.accuracy_score(y_test, ypred_bin_test)
classification_report = sklearn.metrics.classification_report(y_test, ypred_bin_test)

In [80]:
confusion_df = pd.DataFrame(confusion,
                  index=['Actual negative','Actual positive'],
                  columns=['Predicted negative', 'Predicted positive'])

In [91]:
directory = "something_"
text_file = open(directory + "metrics.txt", "w")
text_file.write("AUC: {} \n".format(round(auc,3)))
text_file.write("Accuracy: {} \n\n".format(round(accuracy,3)))
text_file.write(classification_report)
text_file.close()

In [82]:
fitted = pd.DataFrame(index=initial_features)

fitted['coefs'] = lr.coef_[0]

fitted['odds_ratio'] = fitted.coefs.apply(np.exp)

fitted = fitted.sort_values(by='odds_ratio', ascending=False)

fitted.head()

,coefs,odds_ratio
visible_mean,0.066825,1.069109
visible_norm_range,0.063088,1.065120
IR_norm_range,-0.008238,0.991795
entropy_x_contrast,-0.017736,0.982420
IR_mean,-0.060727,0.941080


In [3]:
kwargs = {'tiff':112,
        'brian':421,
         'something':'okiedokie'}

In [8]:
def test(**kwarrgs):
    result = ""
    # Iterating over the Python kwargs dictionary
    for arg in kwargs.values():
        result += arg
    return result

result = test(**kwargs)
print(result)

TypeError: can only concatenate str (not "int") to str

In [6]:
test(**kwargs)

NameError: name 'tiff' is not defined

In [10]:
folder1 = 'folder1'
folder2 = 'folder2'
print(folder1 +'/'+ folder2)

folder1/folder2


In [12]:
root = Path(".")

In [16]:
root / (folder1 + "/" + folder2)

PosixPath('folder1/folder2')

In [18]:
model

NameError: name 'model' is not defined